In [20]:
import h5py    
import numpy as np    
import os
import pandas as pd

In [7]:
files = [f for f in os.listdir('.') if '.h5' in f]

In [29]:
f = h5py.File(files[2],'r+')  
combs = list(f.keys())


In [30]:
f[combs[0]][()].shape

(4000, 2002)